In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [947 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
# Install pytest and pytest-sugar to make our output look nice.
!pip install -q pytest pytest-sugar

In [3]:
# Create and navigate to the tdd directory.
from pathlib import Path
if Path.cwd().name != 'tests':
    %mkdir tests
    %cd tests
# Show the current working directory.  
%pwd

/content/tests


'/content/tests'

In [4]:
# Initialize the __init__.py file. 
# This file will be stored in our pwd (/content/tests)
%%file __init__.py
pass

Writing __init__.py


In [11]:
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession

import time
spark = SparkSession.builder.appName("sparkEnergyData").getOrCreate()

# url = "https://raw.githubusercontent.com/ahakobia/Group4_NFLX_MIDTERM/main/Resources/organised_Gen.csv?token=GHSAT0AAAAAABXH6XNLVR2KDPQKXAWUGPNQYZX5EYQ"
# spark.sparkContext.addFile(url)
# energy = spark.read.csv(SparkFiles.get("organised_Gen.csv"), header=True)

spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
energy = spark.read.csv('organised_Gen.csv', header = True)
energy = energy.withColumnRenamed('TYPE OF PRODUCER', 'producer')
energy = energy.withColumnRenamed('ENERGY SOURCE', 'source')
energy = energy.withColumnRenamed('GENERATION (Megawatthours)', 'generated')
energy.write.parquet('parquet_energy',mode='overwrite')
p_energy = spark.read.parquet('parquet_energy')
p_energy.createOrReplaceTempView('p_energy_data')

start_time = time.time()
transformed_df = spark.sql("""
    SELECT
        YEAR as year,
        MONTH as month,
        STATE as state, producer, source, generated
        FROM p_energy_data
        WHERE state != 'US-TOTAL' AND producer != 'Total Electric Power Industry'
        AND source != 'Total'
        """)
transformed_df

  
q_df = spark.sql("""SELECT year, source, SUM(generated) as Total_generated
            FROM p_energy_data 
            WHERE producer != 'Total Electric Power Industry' 
                AND source != 'Total' 
                AND state != 'US-TOTAL'
                AND year != 2022
            Group By year, source
            Order By year
                """)
q_df


print("--- %s seconds ---" % (time.time() - start_time))

--- 0.056616783142089844 seconds ---


In [12]:
q_df.show()

+----+--------------------+---------------+
|year|              source|Total_generated|
+----+--------------------+---------------+
|2001|         Other Gases|      9039473.0|
|2001|Solar Thermal and...|       542755.0|
|2001|                Coal|  1.903955939E9|
|2001|Wood and Wood Der...|    3.5199905E7|
|2001|          Geothermal|    1.3740501E7|
|2001|       Other Biomass|    1.4548144E7|
|2001|           Petroleum|   1.24880223E8|
|2001|         Natural Gas|   6.39129114E8|
|2001|               Other|    1.1906353E7|
|2001|             Nuclear|   7.68826308E8|
|2001|                Wind|      6737332.0|
|2001|Hydroelectric Con...|   2.16961052E8|
|2001|      Pumped Storage|     -8823445.0|
|2002|          Geothermal|    1.4491312E7|
|2002|             Nuclear|   7.80064087E8|
|2002|           Petroleum|    9.4567379E7|
|2002|Wood and Wood Der...|    3.8665032E7|
|2002|       Other Biomass|    1.5043713E7|
|2002|                Wind|    1.0354277E7|
|2002|                Coal|  1.9

In [13]:
# Create a bank_data.py file and write the function to it. 
# This file will be stored in our pwd (/content/tests).
%%file total_energy.py

# Import findspark() and initialize. 
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkEnergyData").getOrCreate()

# Create the import_data function. 
def import_data():
    spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
    energy = spark.read.csv('organised_Gen.csv', header = True)
    energy = energy.withColumnRenamed('TYPE OF PRODUCER', 'producer')
    energy = energy.withColumnRenamed('ENERGY SOURCE', 'source')
    energy = energy.withColumnRenamed('GENERATION (Megawatthours)', 'generated')
    energy.write.parquet('parquet_energy',mode='overwrite')
    p_energy = spark.read.parquet('parquet_energy')
    p_energy.createOrReplaceTempView('p_energy_data')
    return p_energy

def transform_data():
    transformed_df = spark.sql("""
    SELECT
        YEAR as year,
        MONTH as month,
        STATE as state, producer, source, generated
        FROM p_energy_data
        WHERE state != 'US-TOTAL' AND producer != 'Total Electric Power Industry'
        AND source != 'Total'
        """)
    return transformed_df

def query_data():
  
    q_df = spark.sql("""SELECT year, source, SUM(generated) as Total_generated
            FROM p_energy_data 
            WHERE producer != 'Total Electric Power Industry' 
                AND source != 'Total' 
                AND state != 'US-TOTAL'
                AND year != 2022
            Group By year, source
            Order By year
                """)
    return q_df


Writing total_energy.py


In [16]:
# Create a test_bank_data.py file and write the test functions to it. 
# This file will be stored in our pwd (/content/tests).
%%file test_total_energy.py

# From the bank_data.py file and import the import_data function. 
from total_energy import (import_data, transform_data, query_data)

# Write the tests. 
def test_row_count_before_transform():
  df = import_data()
  assert df.count() == 496774

def test_column_count_before_transform():
  df = import_data()
  assert len(df.columns) == 7

def test_row_count_after_transform():
    df = transform_data()
    assert df.count() != 496774

def test_column_count_after_transform():
    df = transform_data()
    assert len(df.columns) != 7

def test_row_count_query():
    df = query_data()
    assert df.count() == 273

def test_column_count_query():
    df = query_data()
    assert len(df.columns) == 3

Overwriting test_total_energy.py


In [17]:
# Run the test_import_data.py file with pytest. 
!python -m pytest test_total_energy.py

Test session starts (platform: linux, Python 3.7.14, pytest 3.6.4, pytest-sugar 0.9.5)
rootdir: /content/tests, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_total_energy.py ✓✓✓✓✓✓                                     100% ██████████

Results (29.59s):
       6 passed
